In [2]:
!pip install gensim
import json

In [3]:
import nltk
nltk.download("reuters") # if necessary
from nltk.corpus import twitter_samples



[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\jburk\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


In [4]:
with open("../twitter_crawler/static_tweets_outv3.json") as f:
    tweets = json.load(f)
    
print(tweets[:10])

[{'tweet id': '646428042861023232', 'user id': '23620631', 'text': 'Interesting from the AP: Instead of climate change skeptics, use  ‘those who reject mainstream climate science’ http://t.co/ZfOCSwyhlM', 'lang': 'en', 'user location': 'Melbourne, Australia', 'user geo_enabled': True, 'geo': [144.8459012, -37.6700639], 'created at': '2015-09-22 20:57:10', 'hashtags': []}, {'tweet id': '643551600233943040', 'user id': '3244946022', 'text': '@CarlKatter and Tony Abbott was too gutless to face the media.  MT no change on marriage equality, no action on climate change, same old', 'lang': 'en', 'user location': 'Melbourne, Victoria', 'user geo_enabled': True, 'geo': [144.85029343, -37.66833917], 'created at': '2015-09-14 22:27:12', 'hashtags': []}, {'tweet id': '578777154197983232', 'user id': '171765896', 'text': 'Some of my thoughts on #Healthpromotion #climatechange #equity and #HiAP - in 20 minutes! https://t.co/New1zVhGvl. http://t.co/nN5HbfNy82', 'lang': 'en', 'user location': 'world'

In [5]:
from nltk.tokenize.casual import TweetTokenizer

tokenizer = TweetTokenizer(preserve_case=False, reduce_len=True, strip_handles=True)

In [6]:
import re

pattern = re.match

training_set = []
training_classifications = []

test_set = []
test_classifications = []

topic = "acq"

for tweet in tweets:
    tweetTokens = tokenizer.tokenize(tweet['text'])
    training_set.append(tweetTokens)
            
print("Train Size:", len(training_set))

Train Size: 6388


In [7]:
import logging
import operator

import gensim
from gensim import corpora

import nltk

# for gensim to output some progress information while it's training
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

en_stop = set(nltk.corpus.stopwords.words('english'))

def preprocessing(dataset):
    # we filter stopwords using nltk stopword list
    text_data = [[word.lower() for word in doc if (len(word)>4 and word.lower() not in en_stop) ] for doc in dataset]
    # Dictionary encapsulates the mapping between normalized words and their integer ids.
    dictionary = corpora.Dictionary(text_data)
    # no_below: Keep tokens which are contained in at least no_below documents.
    # no_above: Keep tokens which are contained in no more than no_above documents 
    #           (fraction of total corpus size, not an absolute number).
    dictionary.filter_extremes(no_below=10, no_above=0.5)
    # Filter out the 20 most frequent tokens that appear in the documents.
    dictionary.filter_n_most_frequent(20)
    # convert documents to BOW representations
    corpus = [dictionary.doc2bow(doc) for doc in text_data]
    
    return corpus, dictionary

preprocessed_train, train_dictionary = preprocessing(training_set)
#preprocessed_test, test_dictionary = preprocessing(test_set)

c:\users\jburk\appdata\local\programs\python\python38\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
2021-05-22 14:32:50,717 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2021-05-22 14:32:50,827 : INFO : built Dictionary(13161 unique tokens: ['change', 'climate', 'http://t.co/zfocswyhlm', 'instead', 'interesting']...) from 6388 documents (total 52032 corpus positions)
2021-05-22 14:32:50,827 : INFO : Dictionary lifecycle event {'msg': "built Dictionary(13161 unique tokens: ['change', 'climate', 'http://t.co/zfocswyhlm', 'instead', 'interesting']...) from 6388 documents (total 52032 corpus positions)", 'datetime': '2021-05-22T14:32:50.827814', 'gensim': '4.0.1', 'python': '3.8.6 (tags/

In [47]:
num_topics = 3
#alpha = document-topic prior
#eta (beta in lecture) = topic-word prior
model = gensim.models.LdaModel(preprocessed_train, id2word=train_dictionary,
                               num_topics=num_topics, alpha='auto', eta='auto',
                               passes=10)

for topic_id in range(model.num_topics):
    # extract 10 top words for each topic
    topk = model.show_topic(topic_id, 10)
    topk_words = [ w for w, _ in topk ]

    print('{}: {}'.format(topic_id, ' '.join(topk_words)))

2021-05-22 16:07:21,496 : INFO : using autotuned alpha, starting with [0.33333334, 0.33333334, 0.33333334]
2021-05-22 16:07:21,498 : INFO : using serial LDA version on this node
2021-05-22 16:07:21,501 : INFO : running online (multi-pass) LDA training, 3 topics, 10 passes over the supplied corpus of 6388 documents, updating model once every 2000 documents, evaluating perplexity every 6388 documents, iterating 50x with a convergence threshold of 0.001000
2021-05-22 16:07:21,502 : INFO : PROGRESS: pass 0, at document #2000/6388
2021-05-22 16:07:22,981 : INFO : optimized alpha [0.35240626, 0.35705715, 0.34677207]
2021-05-22 16:07:22,981 : INFO : merging changes from 2000 documents into a model of 6388 documents
2021-05-22 16:07:22,981 : INFO : topic #0 (0.352): 0.013*"think" + 0.011*"emissions" + 0.010*"sydney" + 0.009*"health" + 0.009*"issue" + 0.009*"minister" + 0.008*"nothing" + 0.008*"please" + 0.007*"impact" + 0.007*"direct"
2021-05-22 16:07:22,989 : INFO : topic #1 (0.357): 0.016*"#

2021-05-22 16:07:27,086 : INFO : topic #1 (0.510): 0.015*"threat" + 0.014*"urgent" + 0.014*"strong" + 0.013*"first" + 0.012*"planet" + 0.012*"still" + 0.012*"policy" + 0.011*"earth" + 0.011*"fight" + 0.011*"right"
2021-05-22 16:07:27,088 : INFO : topic #2 (0.489): 0.022*"future" + 0.016*"important" + 0.013*"report" + 0.012*"denial" + 0.011*"power" + 0.011*"leaders" + 0.011*"going" + 0.010*"#climatecrisis" + 0.010*"believe" + 0.010*"adaptation"
2021-05-22 16:07:27,089 : INFO : topic diff=0.347265, rho=0.438782
2021-05-22 16:07:27,092 : INFO : PROGRESS: pass 2, at document #2000/6388
2021-05-22 16:07:27,590 : INFO : optimized alpha [0.538094, 0.53407836, 0.5032105]
2021-05-22 16:07:27,590 : INFO : merging changes from 2000 documents into a model of 6388 documents
2021-05-22 16:07:27,590 : INFO : topic #0 (0.538): 0.016*"think" + 0.014*"solve" + 0.013*"health" + 0.013*"australia's" + 0.012*"agenda" + 0.012*"thanks" + 0.011*"emissions" + 0.010*"minister" + 0.010*"support" + 0.009*"impact"


2021-05-22 16:07:29,932 : INFO : topic #1 (0.669): 0.016*"threat" + 0.014*"urgent" + 0.013*"strong" + 0.013*"first" + 0.013*"policy" + 0.013*"planet" + 0.012*"still" + 0.012*"right" + 0.011*"earth" + 0.011*"weather"
2021-05-22 16:07:29,934 : INFO : topic #2 (0.634): 0.023*"future" + 0.016*"important" + 0.014*"going" + 0.012*"report" + 0.012*"power" + 0.012*"denial" + 0.012*"australian" + 0.011*"believe" + 0.011*"security" + 0.010*"leaders"
2021-05-22 16:07:29,936 : INFO : topic diff=0.231648, rho=0.372833
2021-05-22 16:07:29,940 : INFO : PROGRESS: pass 4, at document #2000/6388
2021-05-22 16:07:30,441 : INFO : optimized alpha [0.6936692, 0.69229597, 0.6502877]
2021-05-22 16:07:30,443 : INFO : merging changes from 2000 documents into a model of 6388 documents
2021-05-22 16:07:30,445 : INFO : topic #0 (0.694): 0.017*"think" + 0.015*"australia's" + 0.014*"solve" + 0.014*"health" + 0.014*"agenda" + 0.013*"thanks" + 0.013*"emissions" + 0.012*"support" + 0.011*"minister" + 0.011*"country"
20

2021-05-22 16:07:32,634 : INFO : topic #1 (0.803): 0.016*"threat" + 0.013*"policy" + 0.013*"urgent" + 0.013*"first" + 0.013*"strong" + 0.013*"planet" + 0.012*"still" + 0.012*"right" + 0.011*"weather" + 0.011*"earth"
2021-05-22 16:07:32,636 : INFO : topic #2 (0.764): 0.023*"future" + 0.016*"important" + 0.014*"going" + 0.014*"australian" + 0.012*"power" + 0.012*"believe" + 0.012*"denial" + 0.012*"scientists" + 0.011*"report" + 0.011*"security"
2021-05-22 16:07:32,637 : INFO : topic diff=0.178290, rho=0.329798
2021-05-22 16:07:32,640 : INFO : PROGRESS: pass 6, at document #2000/6388
2021-05-22 16:07:33,023 : INFO : optimized alpha [0.82046604, 0.8247482, 0.77918863]
2021-05-22 16:07:33,023 : INFO : merging changes from 2000 documents into a model of 6388 documents
2021-05-22 16:07:33,023 : INFO : topic #0 (0.820): 0.017*"think" + 0.015*"australia's" + 0.015*"health" + 0.014*"solve" + 0.014*"agenda" + 0.014*"emissions" + 0.013*"thanks" + 0.012*"support" + 0.012*"minister" + 0.011*"country

2021-05-22 16:07:35,196 : INFO : topic #1 (0.912): 0.016*"threat" + 0.014*"policy" + 0.013*"first" + 0.013*"urgent" + 0.013*"planet" + 0.013*"strong" + 0.012*"still" + 0.012*"right" + 0.012*"weather" + 0.011*"report"
2021-05-22 16:07:35,197 : INFO : topic #2 (0.869): 0.023*"future" + 0.015*"important" + 0.015*"australian" + 0.015*"going" + 0.012*"scientists" + 0.012*"believe" + 0.012*"power" + 0.012*"denial" + 0.011*"environment" + 0.011*"security"
2021-05-22 16:07:35,198 : INFO : topic diff=0.149447, rho=0.298887
2021-05-22 16:07:35,201 : INFO : PROGRESS: pass 8, at document #2000/6388
2021-05-22 16:07:35,556 : INFO : optimized alpha [0.91918653, 0.93126136, 0.8823416]
2021-05-22 16:07:35,556 : INFO : merging changes from 2000 documents into a model of 6388 documents
2021-05-22 16:07:35,556 : INFO : topic #0 (0.919): 0.017*"think" + 0.015*"australia's" + 0.015*"health" + 0.015*"emissions" + 0.015*"solve" + 0.014*"agenda" + 0.013*"thanks" + 0.013*"support" + 0.012*"minister" + 0.011*"c

2021-05-22 16:07:37,553 : INFO : topic #1 (0.999): 0.015*"threat" + 0.014*"policy" + 0.013*"first" + 0.013*"planet" + 0.013*"report" + 0.012*"urgent" + 0.012*"strong" + 0.012*"right" + 0.012*"still" + 0.012*"weather"
2021-05-22 16:07:37,555 : INFO : topic #2 (0.952): 0.023*"future" + 0.015*"australian" + 0.015*"important" + 0.015*"going" + 0.013*"scientists" + 0.012*"believe" + 0.012*"power" + 0.012*"denial" + 0.011*"environment" + 0.011*"security"
2021-05-22 16:07:37,556 : INFO : topic diff=0.131821, rho=0.275304
2021-05-22 16:07:37,559 : INFO : LdaModel lifecycle event {'msg': 'trained LdaModel(num_terms=738, num_topics=3, decay=0.5, chunksize=2000) in 16.06s', 'datetime': '2021-05-22T16:07:37.558744', 'gensim': '4.0.1', 'python': '3.8.6 (tags/v3.8.6:db45529, Sep 23 2020, 15:52:53) [MSC v.1927 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}


0: solve think australia's support health emissions thanks agenda water carbonvestment
1: threat policy first planet report urgent strong right still weather
2: future australian important going scientists believe power denial environment security


In [48]:
topicDict = {}

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
for doc, tweet in enumerate(tweets):
    score = sia.polarity_scores(tweet['text'])['compound']
    topics = model.get_document_topics(preprocessed_train[doc])
    for topic, prob in topics:
        if topic in topicDict:
            topicDict[topic]['sentiment'] += score*prob
            topicDict[topic]['tweet_count'] += prob
        else:
            topicDict[topic] = {'sentiment':score*prob,'tweet_count':prob}
print(topicDict)

#print(sia.polarity_scores(tweets[doc]['text']))

#print(model.get_document_topics(preprocessed_train[doc]))
#print("#"*50)

#for index, score in sorted(model[preprocessed_train[doc]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, model.print_topic(index, 10)))

{0: {'sentiment': 79.45373995072771, 'tweet_count': 2120.6467}, 1: {'sentiment': 61.12316884857871, 'tweet_count': 2183.9297}, 2: {'sentiment': 77.14769096040666, 'tweet_count': 2083.4248}}


In [49]:
for topic in topicDict.keys():
    print("Topic %d: %.1f tweets, %.5f average sentiment" %
          (topic, topicDict[topic]['tweet_count'],
           topicDict[topic]['sentiment']/topicDict[topic]['tweet_count'])
         )

Topic 0: 2120.6 tweets, 0.03747 average sentiment
Topic 1: 2183.9 tweets, 0.02799 average sentiment
Topic 2: 2083.4 tweets, 0.03703 average sentiment


In [50]:
topicDict = {}

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
for doc, tweet in enumerate(tweets):
    score = sia.polarity_scores(tweet['text'])['compound']
    topics = model.get_document_topics(preprocessed_train[doc])
    maxProb = 0
    topicAssign = -1
    for topic, prob in topics:
        if prob > maxProb:
            topicAssign = topic
            maxProb = prob
    
    if topicAssign in topicDict:
        topicDict[topicAssign]['sentiment'] += score
        topicDict[topicAssign]['tweet_count'] += 1
    else:
        topicDict[topicAssign] = {'sentiment':score,'tweet_count':1}
print(topicDict)

#print(sia.polarity_scores(tweets[doc]['text']))

#print(model.get_document_topics(preprocessed_train[doc]))
#print("#"*50)

#for index, score in sorted(model[preprocessed_train[doc]], key=lambda tup: -1*tup[1]):
#    print("\nScore: {}\t \nTopic: {}".format(score, model.print_topic(index, 10)))

{1: {'sentiment': 62.515800000000006, 'tweet_count': 2829}, 0: {'sentiment': 91.27999999999975, 'tweet_count': 1975}, 2: {'sentiment': 63.92879999999995, 'tweet_count': 1584}}


In [51]:
for topic in topicDict.keys():
    print("Topic %d: %.1f tweets, %.5f average sentiment" %
          (topic, topicDict[topic]['tweet_count'],
           topicDict[topic]['sentiment']/topicDict[topic]['tweet_count'])
         )

Topic 1: 2829.0 tweets, 0.02210 average sentiment
Topic 0: 1975.0 tweets, 0.04622 average sentiment
Topic 2: 1584.0 tweets, 0.04036 average sentiment


In [54]:

from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()
for doc, tweet in enumerate(tweets):
    topics = model.get_document_topics(preprocessed_train[doc])
    for topic, prob in topics:
        if prob > .6:
            print(topic)
            print(tweet['text'])
            print("#"*50)
    

1
Brad and I can singlehandedly reverse global warming. All we have to do is show up and cold weather ensues. It's 60 F in Melbourne-was &gt;100.
##################################################
2
We don't 'believe in climate change', we accept the evidence that it is occurring (Age article today). Language is important.
##################################################
0
Watch and learn Tony Abbott - social justice, including people's input and climate change matter - stop being the bully or lose as in Vic
##################################################
0
@BarackObama points @TonyAbbottMHR at #ClimateChange,Vlad the Impaler ignores Tone #G20 was about how ineffective our PM is? #auspoI
##################################################
2
Australia hang your head in shame re choosing big business interest over climate change and our future generations
##################################################
2
"@CECHR_UoD: Which co's caused #globalwarming? http://t.co/srERoJHnRm some fu

2
“@DanJCass: .@WillMcGoldrick:#Coal shares plunge ahead of Obama’s climate-change speech http://t.co/xQlNKENRUm”
##################################################
2
When McKibben speaks on @QandA it sounds to me like there is no climate change "debate".  When Bernardi speaks i get scared #QandA
##################################################
0
@preston_f @TonyAbbottMHR the liberal government think #climatechange is a mith &amp; are to lazy to invest in alternative energy #auspol
##################################################
0
Thanks @billshortenmp for standing up for climate change. I think dropping carbon tax only for an Emissions Trading System is great approach
##################################################
0
@whereisdaz Worse, he thinks climate change is "crap".
##################################################
1
@CraigEmersonMP @redroversays of course climate change wasn't an issue in the 1950's, and that's where Tony and the LNP exist. 
#worldisflat
###############

1
@bullfrogr yes but much more when I was younger. Climate change has had a big effect here. The sun is very strong here you burn easily
##################################################
1
@ABCNews24 @frankellyabc see Lateline 08.07 scathing attack by Lord Deben, Chair UK Climate Change C'mtee, on Abbott's lack of concern
##################################################
1
@AdamBandt It's amazing how the seasons change 4 times a year, without need for political blether and additional taxes #climatechange
##################################################
2
Strewth @tanya_plibersek, Andrew Bolt thought you were "just another global warming liar", then he really got going http://t.co/blaKgbOmwJ
##################################################
0
@markgkenny @smh Irrelevant because, as on gun control, Obama's empty words on climate change don't have support of House of Reps or Senate
##################################################
0
There is a need of many lorax in this decade. Let'

1
Tony Abbott is no action man on climate change http://t.co/b3drtp20UV. Outstanding summary of Abbott's nonsense on climate. Dangerous!
##################################################
2
Hey @TonyAbbottMHR sorry to burst your delusional bubble but climate change is manmade and it is the most important thing facing us. #idiot
##################################################
0
Perhaps our grandkids will ask why we built swimming pools under water? @curlcurl #sealevelrise #climatechange #cop https://t.co/Pyo3bhbGTD
##################################################
2
@1001ptsAU do I really care tea party don't believe in climate change bronny flying in helicopters at our cost 10 kill qld roads last week
##################################################
1
Great ‘must read’ on #climatechange news for #marketing #PR 
Web http://t.co/z75o1tMjyT 
Flipboard http://t.co/v5g7a2HSa2 
#reality
##################################################
1
Summary of the latest #climatechange news for #

1
Greg Sheridan: Why don't we look to the Americans on climate change? Nothing ever goes wrong over there, they get everything right! #qanda
##################################################
0
True Abbott, destroying the country #Lnp #ausvotes  http://t.co/l7NMJjjJp2
##################################################
0
Call on @timeshighered Academic Summit organisers to put climate change and #divestment on the agenda TODAY http://t.co/QHLTvnm9mK #THEWAS
##################################################
0
#sydcitytalk business and climate change summit @ Sydney Town Hall Concert Hall https://t.co/LZUAFAGI0k
##################################################
1
We've been going on as a nation foolishly thinking the scientific certainty of #climate change won't impact us ... But that's so deluded!
##################################################
1
2 degrees C or higher global warming is terrible for the whole of Australia ... We must mitigate pollution and adapt! #climate
###########

...

0
Fine morning to be at #ClimateFizza with wentworth community calling on @TurnbullMalcolm to get serious about #climatechange #ausvotes
##################################################
0
Absolutely renewables. A rapid reduction in gas will have instant positive impacts. Gas’ Global Warming Potential i… https://t.co/gT45gina73
##################################################
0
RT @chaser: "Lattes won't solve climate change" says man who thinks milkshakes will solve sexual assault
##################################################
0
RT @climateactiontr: THREAD: Projected end of century warming from #ParisAgreement pledges has dropped by 0.2˚C to 2.4˚C: our latest analys…
##################################################
0
RT @chaser: "Lattes won't solve climate change" says man who thinks milkshakes will solve sexual assault
##################################################
2
RT @JohnRHewson: Surely the risks to our national security of climate change are much greater than the ris

0
RT @CFigueres: Unprecedented preparation of nat'l #climateaction plans, covering 87% of global GHG http://t.co/FUtmnMoNZW #COP21 http://t.c…
##################################################
0
RT @CFigueres: #More #climateaction commitments showcased at #SB42 in Bonn and here: http://t.co/0NGXbLcKuY #COP21 #LPAA #Action2015 http:/…
##################################################
1
RT @ClimateReality: The threat of climate change has never been more urgent. Join the #RoadtoParis: http://t.co/nFUMUs5GJQ #ClimateFact htt…
##################################################
0
RT @ClimateGroup: Are we reaching a positive climate change tipping point? http://t.co/X7ExljAicm via @guardian #cleanrevolution http://t.c…
##################################################
1
RT @Thoriqibrahim: SIDS are the frontline States for most of the threat from climate change. AOSIS want a more specific goals enshrined in …
##################################################
1
RT @nytclimate: Climate chan

1
Moving on to the challenge of managing refugia from #climatechange  in the #murraydarling with @JM_Bennett funded b… https://t.co/kTp0VufVM1
##################################################
1
Why we need a Climate Act Now https://t.co/HZQecpfKLX 
#climateactnow #auspol #ClimateChange #ClimateEmergency #ClimateActionNow
##################################################
0
I just signed up to support the introduction of the Climate Change Bill into the #Australian Parliament. See detail… https://t.co/fAiz00okGR
##################################################
0
RT @chaser: "Lattes won't solve climate change" says man who thinks milkshakes will solve sexual assault
##################################################
1
@colnago70 @kathy_skidmore1 @QandA Yep. Still waiting for a climate change problem to show up
##################################################
2
@WohYeahWohYeah @TiggerTwo4 Only idiots believe the climate change nonsense. It’s not science you believe. You just don’t r

1
#PopeFrancis has said "history will judge" climate change deniers. #CardinalGeorgePell refutes climate change scien… https://t.co/ouZLXFlnWV
##################################################
0
RT @benandjerrysUK: Hey @PritiPatel we think the real crisis is our lack of humanity for people fleeing war, climate change and torture. We…
##################################################
0
RT @sizeofwales: On #Teachersday2020 we’d like to thank you for continuing to raise awareness on climate change during this difficult time.…
##################################################
2
Would recommend these key measures for an effective @WelshGovernment climate change strategy -  set out by… https://t.co/x7Oy4zKfVM
##################################################
0
RT @GAIGriffith: @FemInt shares her views on the imperative of placing women at the helm of action related to #climatechange #policydevelop…
##################################################
1
Reading: US Climate Diplomacy under B

0
RT @pedalbrisbane: Thank you @BarackObama for pushing #climate action. We are trying too! Even if leaders aren't. #g20 #g20brisbane #bne ht…
##################################################
1
'Cut-off grade will be affected by how much global warming gas you are generating. It is a rude shock for the minin… https://t.co/ukpmK3sEFy
##################################################
0
RT @Independent: And the 'worst industrial country in the world for climate change' is...
http://t.co/FKJFHFk13k http://t.co/YUJIrNamh4
##################################################
0
RT @chaser: "Lattes won't solve climate change" says man who thinks milkshakes will solve sexual assault
##################################################
0
RT @chaser: "Lattes won't solve climate change" says man who thinks milkshakes will solve sexual assault
##################################################
1
RT @abcnews: History made as millennial wins lawsuit against superannuation fund for not doing enough on 